## Semantic Kernel: Ramp-Up based on SK's Documentation

To get the latest version of SK Python package, use:

``` bash
pip install --upgrade semantic-kernel
```

## 📒 Notebook 2: AI Services

### 🪜 Step 1: Configure environment

In [1]:
# Import required packages
import os

from semantic_kernel import Kernel
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings

In [2]:
# Set Azure OpenAI backend variables
AOAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_API_DEPLOY")
AOAI_ENDPOINT = os.getenv("AZURE_OPENAI_API_BASE")
AOAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

### 🪜 Step 2: Add AI service

In [3]:
# Initialise kernel
kernel = Kernel()

In [4]:
# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    deployment_name = AOAI_DEPLOYMENT,
    endpoint = AOAI_ENDPOINT,
    api_version = AOAI_API_VERSION,
    service_id = "azure_openai_chat",
)

kernel.add_service(chat_completion)

### 🪜 Step 3: Retrieve AI service

In [5]:
# Retrieve the chat completion service by type
chat_completion_service = kernel.get_service(type=ChatCompletionClientBase)
chat_completion_service.ai_model_type

<OpenAIModelTypes.CHAT: 'chat'>

In [6]:
# Retrieve the chat completion service by id
chat_completion_service = kernel.get_service(service_id="azure_openai_chat")
chat_completion_service.ai_model_type

<OpenAIModelTypes.CHAT: 'chat'>

### 🪜 Step 4: Direct use of Chat Completion services

You can use the chat completion service directly, without adding to the kernel.

In [7]:
# Instantiatte execution settings
execution_settings = OpenAIChatPromptExecutionSettings()

### a) Non-streaming chat completion

In [8]:
# Reset chat history
chat_history = ChatHistory()
chat_history.add_user_message("Can you prepare a short (3 paragraphs) story about red pandas?")

In [9]:
# Get non-streaming response
response = await chat_completion_service.get_chat_message_content(
    chat_history = chat_history,
    settings = execution_settings,
)

print(response)

Deep within the misty forests of the Eastern Himalayas, a small red panda named Tashi darted gracefully through the branches. With its fiery red fur blending seamlessly with the autumn leaves, Tashi was a master of camouflage. Every day, it foraged for bamboo shoots and sweet berries, using its sharp claws to cling effortlessly to the towering trees. Despite its shy nature, Tashi was curious about the world beyond the treetops and often paused to watch birds and butterflies flit by.

One crisp morning, as sunlight filtered through the canopy, Tashi stumbled upon a tiny injured bird nestled among the leaves. The bird’s wing was bent awkwardly, and it chirped weakly for help. Tashi gently nudged the bird onto its back and decided to take it to a nearby clearing where the sunlight was warm and inviting. There, Tashi carefully tended to the bird, sharing some berries and keeping it safe from predators. Over the next few days, the red panda and the bird became unlikely friends, bound by kin

### b) Streaming chat completion

In [10]:
# Reset chat history
chat_history = ChatHistory()
chat_history.add_user_message("Can you prepare a short (3 paragraphs) story about red pandas?")

In [11]:
# Get streaming response
response = chat_completion_service.get_streaming_chat_message_content(
    chat_history = chat_history,
    settings = execution_settings,
)

async for chunk in response:
    print(chunk, end="")

In the misty forests of the Eastern Himalayas, a small red panda named Tashi scurried through the dense bamboo thickets. With her reddish-brown fur blending perfectly with the autumn foliage, she moved gracefully from branch to branch, searching for tender bamboo shoots—their main source of food. Despite her diminutive size, Tashi was an expert climber, using her sharp claws and long, bushy tail for balance as she navigated the towering trees.

One day, while exploring a new part of the forest, Tashi stumbled upon a baby bird that had fallen from its nest. Although red pandas are solitary creatures, her gentle nature compelled her to help. Carefully picking up the fragile bird with her paws, she climbed to a nearby branch and nudged the nestdoor open. After a few tries, she managed to place the chick safely back with its family, feeling a little warmer inside despite the crisp mountain air.

As twilight descended, the forest came alive with the soft sounds of crickets and rustling leav